<a href="https://colab.research.google.com/github/ssnirgudkar/UNet/blob/main/IR_7classes_train_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# will get raw and segmented images from segments.ai. I am not changing the size of the image for now
!pip install --upgrade segments-ai

In [ ]:
from segments import SegmentsClient
api_key = "a89182567b17766b91773021b18d04574cd75109"
client = SegmentsClient(api_key)

In [ ]:
# get the release file
dataset_identifier = "ssnirgudkar/IR-2020-10-22-17-33-25-0"
name = "V2.0"
release = client.get_release(dataset_identifier, name)
print(release)

{'uuid': '13189c97-ebd6-4b06-a2dd-779e7bf5b81f', 'name': 'V2.0', 'description': 'Labels are finalized', 'release_type': 'JSON', 'attributes': {'url': 'https://segmentsai-prod.s3.eu-west-2.amazonaws.com/releases/13189c97-ebd6-4b06-a2dd-779e7bf5b81f.json'}, 'status': 'SUCCEEDED', 'status_info': '', 'created_at': '2021-06-19T18:45:20.180168Z', 'samples_count': 62}


In [7]:
# export the release. This exports both the raw image and the labeled image 
from segments import SegmentsDataset
from segments.utils import export_dataset
dataset = SegmentsDataset(release, labelset='ground-truth', filter_by=['labeled', 'reviewed'])

# Export to COCO panoptic format
export_dataset(dataset, export_format='coco-panoptic')


 42%|████▏     | 26/62 [00:00<00:00, 219.28it/s]

Initializing dataset...
Preloading all samples. This may take a while...


  5%|▍         | 3/62 [00:00<00:02, 23.85it/s]

Initialized dataset with 62 images.
Exporting dataset. This may take a while...


100%|██████████| 62/62 [00:02<00:00, 22.32it/s]

Exported to ./export_coco-panoptic_ssnirgudkar_IR-2020-10-22-17-33-25-0_V2.0.json. Images and labels in segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0


('./export_coco-panoptic_ssnirgudkar_IR-2020-10-22-17-33-25-0_V2.0.json',
 'segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0')

Because the trainning code is for classes and different image size, I will create new training code here. However i cannot do that until i convert the pixel values from ids to category ids. Idea is that I train sing 7 classes, and then i can use that model to auto segment the other images


In [9]:
# read the downloaded files 

import os
import numpy as np
import cv2

input_dir = "/content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0" # directory containing input raw & labeled images
img_size = (512, 640) #it's height and width. it's a 1/2 sized image from the original image from segment.ai
num_classes = 7
batch_size = 5

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".png") and fname not in ("ground-truth")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith("ground-truth.png")  
    ]
)

#prints how many files are in the batch
print("Number of samples:", len(input_img_paths))

#prints the name of 10 sets of input and labeled file
for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)

#below code checks if the labels are in the 1 through 3 range (4 classes - sky,water,object,background). 
#Because if not, then model training gives a NAN error.
#it displays the label file that has an error, and the values of the label
ideal_label_array = [0,1,2,3,4,5,6] # depends on the number of classes
flag = 0 
for target_path in (target_img_paths):
    target_label_array = (np.unique(cv2.imread(target_path,cv2.IMREAD_UNCHANGED)))
    if(set(target_label_array).issubset(set(ideal_label_array))):
       flag +=1
       #print("good label", flag)
    else:    
       flag +=1
       print("Error in label", flag,target_path,target_label_array )  

Number of samples: 186
/content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388082.940481.png | /content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388082.940481_label_ground-truth.png
/content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388082.940481_label_ground-truth.png | /content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388099.294270_label_ground-truth.png
/content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388082.940481_label_ground-truth_coco-panoptic.png | /content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388115.497458_label_ground-truth.png
/content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388099.294270.png | /content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388165.895730_label_ground-truth.png
/content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388099.294270_label_ground-truth.png | /content/segments/ssnirgudkar_IR-2020-10-22-17-33-25-0/V2.0/1603388188.478616_labe